Домашнее задание по лекции <<Чистка данных>>, Хрыльчено Кирилл.

In [1]:
import numpy as np, pandas as pd

Итак, будем оптимизировать/эффективно переписывать некоторые выражения из лекции.
Пример первый --- с суммами.
Для начала, надо создать искусственную выборку.

In [2]:
prices = [str(number) + '$'  for number in np.random.choice(a = range(10000), size = 1500000)]
df = pd.DataFrame(prices, columns = ['Price'])

In [3]:
df.head()

,Price
0,2448$
1,6374$
2,9269$
3,2056$
4,6532$


In [4]:
%%time
df['Price_num'] = df['Price'].apply(lambda x: int(x[:-1]))

CPU times: user 652 ms, sys: 59.9 ms, total: 712 ms
Wall time: 710 ms


Получили время выполнения примерно 700 ms. Попробуем улучшить данный результат:

In [6]:
%%time
df['Price_num2'] = df['Price'].apply(lambda x: x[:-1]).astype(int)

CPU times: user 394 ms, sys: 43.7 ms, total: 437 ms
Wall time: 436 ms


In [7]:
df.head()

,Price,Price_num,Price_num2
0,2448$,2448,2448
1,6374$,6374,6374
2,9269$,9269,9269
3,2056$,2056,2056
4,6532$,6532,6532


Замечательно, получили улучшение по скорости почти в ДВА раза. А в чем тут дело? Видимо, приводить к int'у по отдельности каждую строку более затратно, чем привести к int'у их все в пандасе одним методом.

На том же слайде производится преобразование дат. Посмотрим, получится ли сделать это быстрее.

Генерация данных:

In [8]:
months = range(1, 13)
days = range(1, 31) # пусть будет 30 дней в каждом месяце, не суть важно
years = range(1996, 2019)

dates = [str(np.random.choice(months)) + '.' + str(np.random.choice(days)) + '.' + str(np.random.choice(years)) 
         for i in range(100000)]

df = pd.DataFrame(dates, columns = ['date'])

In [9]:
df.head()

,date
0,3.17.2012
1,8.2.2015
2,11.23.2014
3,12.19.2017
4,6.23.1996


In [10]:
%%time
df['edited_date_1'] = pd.to_datetime(df['date'], errors = 'coerce', yearfirst = False)

CPU times: user 6.78 s, sys: 0 ns, total: 6.78 s
Wall time: 6.81 s


In [11]:
%%time
df['date_edited_2'] = pd.to_datetime(df['date'], errors = 'coerce', format = "%m.%d.%Y", yearfirst = False)

CPU times: user 166 ms, sys: 0 ns, total: 166 ms
Wall time: 171 ms


Получаем улучшение в почти *36 раз*. Формат даты все-таки лучше указывать!

In [12]:
df.head()

,date,edited_date_1,date_edited_2
0,3.17.2012,2012-03-17,2012-03-17
1,8.2.2015,2015-08-02,2015-08-02
2,11.23.2014,2014-11-23,2014-11-23
3,12.19.2017,2017-12-19,2017-12-19
4,6.23.1996,1996-06-23,1996-06-23


Сгенерируем данные с давлением:

In [13]:
measures = [str(np.random.choice(range(100, 200))) + '/' + str(np.random.choice(range(80, 100))) for i in range(1000000)]
df = pd.DataFrame(measures, columns = ['Давление'])

In [14]:
df.head()

,Давление
0,198/89
1,194/88
2,195/94
3,148/94
4,111/82


В слайдах есть вопрос как переписать следующий кусок кода короче (эффективней):

In [16]:
%%time
tmp = df['Давление'].str.split('/')
df['в. давл.'] = tmp.apply(lambda x: x[0])
df['н. давл.'] = tmp.apply(lambda x: x[1])

CPU times: user 1.13 s, sys: 52.1 ms, total: 1.18 s
Wall time: 1.18 s


In [17]:
df.head()

,Давление,в. давл.,н. давл.
0,198/89,198,89
1,194/88,194,88
2,195/94,195,94
3,148/94,148,94
4,111/82,111,82


Далее следует мой способ:

In [19]:
df = pd.DataFrame(measures, columns = ['Давление'])

In [20]:
%%time
df[['в. давл.', 'н. давл.']] = pd.DataFrame(df['Давление'].map(lambda x: x.split('/')).values.tolist())

CPU times: user 896 ms, sys: 68.1 ms, total: 964 ms
Wall time: 962 ms


In [21]:
df.head()

,Давление,в. давл.,н. давл.
0,198/89,198,89
1,194/88,194,88
2,195/94,195,94
3,148/94,148,94
4,111/82,111,82


О чудо! Получилось. Да и работает быстрее. Правда пришлось делать pd.DataFrame(...), т.к. иначе выдает ошибку.
Ну и если немного придираться, мы в итоге разделили давление на два значения, поместили по разным столбцам, НО: оставили строковые значения, хотя по-хорошему надо было бы преобразовать в int :)

Теперь попробуем более эффективно написать код для замены пропусков (тоже задание из слайдов). Сначала генерация данных:

In [24]:
df = pd.DataFrame(np.random.choice(a = range(60, 100), size = (6, 4)), columns = ['площадь', 'площадь 1', 'площадь 2', 'площадь 3'])
df['data'] = ['train', 'train', 'train', 'test', 'test', 'test']
df.loc[1, 'площадь'] = np.nan
df.loc[4, 'площадь'] = np.nan

In [25]:
df.head()

,площадь,площадь 1,площадь 2,площадь 3,data
0,74.0,77,64,68,train
1,NaN,64,80,69,train
2,89.0,66,82,75,train
3,62.0,98,80,79,test
4,NaN,65,72,81,test


In [23]:
# эту ячейку вопроизводить не надо, тут все хорошо.
# вариант 1 - среднее по всему
df['площадь'].fillna(df['площадь'].mean(), inplace = True)

# вариант 2 - среднее по трейну
df['площадь'].fillna(df[df['data'] == 'train']['площадь'].mean(), inplace = True)

Третий вариант, который воспроизведен ниже, вполне можно записать оптимальней:

In [26]:
# вариант 3 - в трейне среднее по трейну, в тесте среднее по тесту
df.loc[df['data'] == 'train', 'площадь'] =\
        df[df['data'] == 'train']['площадь'].fillna(df[df['data'] == 'train']['площадь'].mean())
df.loc[df['data'] == 'test', 'площадь'] =\
        df[df['data'] == 'test']['площадь'].fillna(df[df['data'] == 'test']['площадь'].mean())

In [27]:
df

,площадь,площадь 1,площадь 2,площадь 3,data
0,74.0,77,64,68,train
1,81.5,64,80,69,train
2,89.0,66,82,75,train
3,62.0,98,80,79,test
4,79.0,65,72,81,test
5,96.0,72,98,98,test


У нас есть замечательный метод transform:

In [28]:
# перепишем третий вариант

df['площадь'] = df.groupby("data")['площадь'].transform(lambda x: x.fillna(x.mean()))

In [29]:
df

,площадь,площадь 1,площадь 2,площадь 3,data
0,74.0,77,64,68,train
1,81.5,64,80,69,train
2,89.0,66,82,75,train
3,62.0,98,80,79,test
4,79.0,65,72,81,test
5,96.0,72,98,98,test


Спасибо за внимание. Больше необязательных вопросов про оптимальное переписывание кода нет, так что на этом все!